In [1]:
import os
os.getcwd()

'/home/jupyter/NLP/CPR/Aug 2024'

In [2]:
import vertexai
from vertexai.language_models import TextGenerationModel

2024-09-09 09:55:47.964178: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import pandas as pd
data = pd.read_csv(r'./input/CPR_Aug_2024.csv')
data

,id,text,month,year
0,R_1QXxniNV3YpqUPq,It was about new vehicle,Aug,2024
1,R_3DhVOZ3PTKZkWgp,Yes I have seen commercials for ford,Aug,2024
2,R_5Ea54NwgY2tkvR6,buy a ford product,Aug,2024
3,R_1YEbWB9AKWFLJnK,largest employer in usa,Aug,2024
4,R_1co47Ec7FvuwRpO,Fords recall of F-150 vehicles,Aug,2024
...,...,...,...,...
467,R_6nCY5c1j1ZGUX9S,that it was exapanding,Aug,2024
468,R_3t9fVWP70zFU9iP,It has a good fit for motor that make a good f...,Aug,2024
469,R_1n0P0NNv5Ijvesp,Great,Aug,2024
470,R_3P3eupNFhLpNxFT,No trucks out,Aug,2024


In [4]:
data['text_n'] = data.text.where(data.text.str.endswith('.'),  data.text + '.')
data.head()

,id,text,month,year,text_n
0,R_1QXxniNV3YpqUPq,It was about new vehicle,Aug,2024,It was about new vehicle.
1,R_3DhVOZ3PTKZkWgp,Yes I have seen commercials for ford,Aug,2024,Yes I have seen commercials for ford.
2,R_5Ea54NwgY2tkvR6,buy a ford product,Aug,2024,buy a ford product.
3,R_1YEbWB9AKWFLJnK,largest employer in usa,Aug,2024,largest employer in usa.
4,R_1co47Ec7FvuwRpO,Fords recall of F-150 vehicles,Aug,2024,Fords recall of F-150 vehicles.


In [5]:
#summarization Jan
vertexai.init(project="ford-180395bd732cdd9af050c1f7", location="us-central1")
parameters = {
    "candidate_count": 1,
    "max_output_tokens": 2048,
    "temperature": 0.2,
    "top_p": 1
}
model = TextGenerationModel.from_pretrained("text-bison-32k")

#prompt_train ="""Below are the comments received in a survey where people were asked to mention about the latest news that they heard about Ford Motor Company. Please provide key topics identified and a detailed summary of each topic:"""
prompt_train ="""Below are the comments received in a survey where people were asked about the latest news that they heard about Ford Motor Company. Identify key topics. Provide a detailed summary of each topic. Use present continuous tense for the summary. :"""

data_text = data["text_n"]
data_text_str = data_text.to_string(index=False)
prompt = prompt_train + data_text_str
response = model.predict(prompt,**parameters)
print(f"Response from Model: {response.text}")

Response from Model:  **New Vehicles**
Ford Motor Company is introducing new vehicles, including the 2025 Ford Raptor, the all-electric F-150 Lightning, and the Mustang Mach-E. These vehicles offer innovative features, improved performance, and a focus on sustainability.

**Recalls**
Ford has issued several recalls for various models, including the F-150, Bronco, and Explorer. These recalls address issues such as airbag malfunctions, steering problems, and door latch defects. Customers are advised to check for recall notices and take appropriate action to ensure their safety.

**Electric Vehicles**
Ford is investing heavily in electric vehicle (EV) development and production. The company aims to transition to an all-electric lineup by 2030. Ford's EV offerings include the Mustang Mach-E, E-Transit van, and F-150 Lightning. However, the company is facing challenges in ramping up EV production and managing costs.

**Financial Performance**
Ford's financial performance has been mixed, wit

In [6]:
#Input prompt data
prompt_df = pd.read_csv(r'./input/training_data_Aug.csv')
prompt_df

,Text,Sub_category,Sub_category_names,Category
0,Ford Motor Company is introducing new vehicles...,New Vehicles,New Vehicles,Product related
1,Ford has issued several recalls for various mo...,Recalls,Recalls,Product related
2,Ford is investing heavily in electric vehicle ...,Electric Vehicles,Electric Vehicles,Electrification
3,"Ford's financial performance has been mixed, w...",Financial Performance,Financial Performance,Organization feedback
4,Ford vehicles are generally regarded as reliab...,Product Quality and Reliability,Product Quality and Reliability,Product related
5,Ford is investing in research and development ...,Innovation and Technology,Innovation and Technology,"New technology, unique features or services"
6,Ford is committed to reducing its environmenta...,Environmental Sustainability,Environmental Sustainability,Organization feedback
7,Ford has faced challenges in its labor relatio...,Labor Relations,Labor Relations,Corporate issues
8,Ford enjoys a strong brand reputation and cust...,Brand Perception and Reputation,Brand Perception and Reputation,Branding


In [7]:
#topic prediction
vertexai.init(project="ford-180395bd732cdd9af050c1f7", location="us-central1")
parameters = {
    "candidate_count": 1,
    "max_output_tokens": 5,
    "temperature": 0.2,
    "top_p": 0.8,
    "top_k": 1
}
model = TextGenerationModel.from_pretrained("text-bison@001")

def build_prompt(df):
    catg = df["Sub_category_names"].unique()
    prompt_build = """What is the topic for a given comment?
Options:"""
    prompt_build = prompt_build + "\n-" + "\n-".join(catg)
    prompt_build = prompt_build + "\n\nUse your intelligence in categorizing the comments according to the Examples given and incase the text does not fit in any of the above mentioned categories then assign category as Other. Understand the meanings of the Text first before assigning Categories to the comments.\n" + "\nExamples:\n" 
    for index, row in df.iterrows():
        prompt_build = prompt_build + "\nText: " + row["Text"] + "\nCategories: " + row["Sub_category_names"]+"\n"
    #prompt_build = prompt_build + "\nText:"
    return(prompt_build)

prompt_train = build_prompt(prompt_df)
print(prompt_train)
prompt_cat = "\nCategories: "

import time
def split_data_cat(sub_data):
    sub_category=[]
    for i in range(0,len(sub_data)):
        prompt_test = sub_data['text'].iloc[i]
        prompt_test_s = str(prompt_test)
        prompt = prompt_train + prompt_test_s + prompt_cat
        response = model.predict(prompt,**parameters)
        sub_category.append(response.text)
        time.sleep(2)
    
    final = pd.concat([sub_data,pd.DataFrame(sub_category, columns=['Sub_category'])], axis=1)
    return final


What is the topic for a given comment?
Options:
-New Vehicles
-Recalls
-Electric Vehicles
-Financial Performance
-Product Quality and Reliability
-Innovation and Technology
-Environmental Sustainability
-Labor Relations
-Brand Perception and Reputation

Use your intelligence in categorizing the comments according to the Examples given and incase the text does not fit in any of the above mentioned categories then assign category as Other. Understand the meanings of the Text first before assigning Categories to the comments.

Examples:

Text: Ford Motor Company is introducing new vehicles, including the 2025 Ford Raptor, the all-electric F-150 Lightning, and the Mustang Mach-E. These vehicles offer innovative features, improved performance, and a focus on sustainability.
Categories: New Vehicles

Text: Ford has issued several recalls for various models, including the F-150, Bronco, and Explorer. These recalls address issues such as airbag malfunctions, steering problems, and door latch d

In [8]:
#output = split_data_cat(data_sub)
output = split_data_cat(data)

In [9]:
output

,id,text,month,year,text_n,Sub_category
0,R_1QXxniNV3YpqUPq,It was about new vehicle,Aug,2024,It was about new vehicle.,New Vehicles
1,R_3DhVOZ3PTKZkWgp,Yes I have seen commercials for ford,Aug,2024,Yes I have seen commercials for ford.,Other
2,R_5Ea54NwgY2tkvR6,buy a ford product,Aug,2024,buy a ford product.,Other
3,R_1YEbWB9AKWFLJnK,largest employer in usa,Aug,2024,largest employer in usa.,Other
4,R_1co47Ec7FvuwRpO,Fords recall of F-150 vehicles,Aug,2024,Fords recall of F-150 vehicles.,Recalls
...,...,...,...,...,...,...
467,R_6nCY5c1j1ZGUX9S,that it was exapanding,Aug,2024,that it was exapanding.,Other
468,R_3t9fVWP70zFU9iP,It has a good fit for motor that make a good f...,Aug,2024,It has a good fit for motor that make a good f...,Other
469,R_1n0P0NNv5Ijvesp,Great,Aug,2024,Great.,Other
470,R_3P3eupNFhLpNxFT,No trucks out,Aug,2024,No trucks out.,Other


In [11]:
df_join=pd.merge(output, prompt_df[['Sub_category','Category']], on='Sub_category', how='left')

In [12]:
df_join

,id,text,month,year,text_n,Sub_category,Category
0,R_1QXxniNV3YpqUPq,It was about new vehicle,Aug,2024,It was about new vehicle.,New Vehicles,Product related
1,R_3DhVOZ3PTKZkWgp,Yes I have seen commercials for ford,Aug,2024,Yes I have seen commercials for ford.,Other,NaN
2,R_5Ea54NwgY2tkvR6,buy a ford product,Aug,2024,buy a ford product.,Other,NaN
3,R_1YEbWB9AKWFLJnK,largest employer in usa,Aug,2024,largest employer in usa.,Other,NaN
4,R_1co47Ec7FvuwRpO,Fords recall of F-150 vehicles,Aug,2024,Fords recall of F-150 vehicles.,Recalls,Product related
...,...,...,...,...,...,...,...
467,R_6nCY5c1j1ZGUX9S,that it was exapanding,Aug,2024,that it was exapanding.,Other,NaN
468,R_3t9fVWP70zFU9iP,It has a good fit for motor that make a good f...,Aug,2024,It has a good fit for motor that make a good f...,Other,NaN
469,R_1n0P0NNv5Ijvesp,Great,Aug,2024,Great.,Other,NaN
470,R_3P3eupNFhLpNxFT,No trucks out,Aug,2024,No trucks out.,Other,NaN


In [14]:
# Add functionality: If Sub_category is 'Other', assign Category as 'Other' as well
df_join['Category'] = df_join.apply(lambda row: 'Other' if row['Sub_category'] == 'Other' else row['Category'], axis=1)
df_join

,id,text,month,year,text_n,Sub_category,Category
0,R_1QXxniNV3YpqUPq,It was about new vehicle,Aug,2024,It was about new vehicle.,New Vehicles,Product related
1,R_3DhVOZ3PTKZkWgp,Yes I have seen commercials for ford,Aug,2024,Yes I have seen commercials for ford.,Other,Other
2,R_5Ea54NwgY2tkvR6,buy a ford product,Aug,2024,buy a ford product.,Other,Other
3,R_1YEbWB9AKWFLJnK,largest employer in usa,Aug,2024,largest employer in usa.,Other,Other
4,R_1co47Ec7FvuwRpO,Fords recall of F-150 vehicles,Aug,2024,Fords recall of F-150 vehicles.,Recalls,Product related
...,...,...,...,...,...,...,...
467,R_6nCY5c1j1ZGUX9S,that it was exapanding,Aug,2024,that it was exapanding.,Other,Other
468,R_3t9fVWP70zFU9iP,It has a good fit for motor that make a good f...,Aug,2024,It has a good fit for motor that make a good f...,Other,Other
469,R_1n0P0NNv5Ijvesp,Great,Aug,2024,Great.,Other,Other
470,R_3P3eupNFhLpNxFT,No trucks out,Aug,2024,No trucks out.,Other,Other


In [16]:
df_join.to_csv('./output/topics_cpr_Aug.csv')